<a href="https://colab.research.google.com/github/RifaldiAchmad/Maxim-Sentiment-Analysis/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import json

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [195]:
!git clone https://github.com/RifaldiAchmad/Maxim-Sentiment-Analysis.git

fatal: destination path 'Maxim-Sentiment-Analysis' already exists and is not an empty directory.


In [196]:
seed = 0
np.random.seed(seed)

In [197]:
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.read_excel('/content/Maxim-Sentiment-Analysis/maxim_training.xlsx')
app_reviews_df.head()

,sentiment,content,at
0,Negatif,Pencariannya lama banget padahal banyak yg online,2025-02-28 23:45:49
1,Positif,slalu pakai maxim,2025-02-28 23:00:33
2,Positif,bagus pelayanan nya orangnya ramah,2025-02-28 22:55:05
3,Negatif,"Nyari alamat susah banget, udah ku tulis sejel...",2025-02-28 22:52:00
4,Positif,asik lah pokoknya..jangan kapok ya bang,2025-02-28 22:50:23


In [198]:
app_reviews_df.shape

(34831, 3)

In [199]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34831 entries, 0 to 34830
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sentiment  34831 non-null  object        
 1   content    34830 non-null  object        
 2   at         34831 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 816.5+ KB


In [200]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = app_reviews_df.drop_duplicates()

In [201]:
# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
clean_df.shape

(34831, 3)

In [202]:
clean_df['sentiment'].value_counts()

,count
sentiment,
Positif,26237
Negatif,8560
Netral,34


## **Preprocessing Text**

In [203]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [204]:
def filteringText(text):
    # Daftar stopwords manual Bahasa Indonesia beserta variasi singkatannya
    stopwords_id = {
        "yang", "yg",
        "di",
        "ke",
        "dari", "dr",
        "dan",
        "atau",
        "itu",
        "ini",
        "untuk", "utk",
        "pada", "pd",
        "sebagai", "sebgai",
        "dengan", "dgn",
        "oleh", "olh",
        "karena", "krn",
        "jika", "jka", "jk",
        "tapi", "tpi", "tp",
        "namun",
        "jadi", "jdi", "jd",
        "saja", "aja", "aj",
        "adalah", "adl",
        "pula",
        "saat",
        "telah", "tlh",
        "akan",
        "bisa", "bs",
        "mereka",
        "kami",
        "kita",
        "anda",
        "tersebut", "tsb",
        "seperti", "sprti", "spt",
        "sekarang", "skrg",
        "masih",
        "agar",
        "tentang", "ttg",
        "selain",
        "antara",
        "kini",
        "demikian",
        "masing",
        "begitu",
        "sesuai",
        "hanya",
        "adapun",
        "lagi", "lg",
        "nya", "yaa", "loh", "kah", "woi", "woii", "woy",
        "klo",
        "sblm",
        "dg",
        "dpt",
        "wkwk",
        "saya",
        "buat"
    }
    # Hanya menghapus kata yang ada dalam daftar stopwords (perbandingan dengan huruf kecil)
    filtered = [word for word in text if word.lower() not in stopwords_id]
    return filtered

In [205]:
try:
    with open('/content/Maxim-Sentiment-Analysis/slang_words_id.txt', "r", encoding="utf-8") as f:
        slangwords_str = f.read().strip()  # Membaca file sebagai string

        if slangwords_str:
            slangwords = json.loads(slangwords_str)  # Konversi dari string ke dictionary
            print("Berhasil dikonversi ke dictionary!")
            print(list(slangwords.items())[:5])  # Menampilkan 5 contoh pertama
        else:
            print("File kosong!")
except FileNotFoundError:
    print("File tidak ditemukan! Periksa lokasi path-nya.")
except json.JSONDecodeError:
    print("Format JSON tidak valid! Periksa isi file.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Berhasil dikonversi ke dictionary!
[('@', 'di'), ('abis', 'habis'), ('ad', 'ada'), ('adlh', 'adalah'), ('afaik', 'as far as i know')]


In [206]:
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [207]:
clean_df.dropna(subset=['content'], inplace=True)

In [208]:
clean_df['text_clean'] = clean_df['content'].apply(cleaningText) # Membersihkan teks
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText) # Mengubah huruf dalam teks menjadi huruf kecil
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords) # Mengganti kata-kata slang dengan kata-kata standar
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText) # Memecah teks menjadi token (kata-kata)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText) # Menghapus kata-kata stop (kata-kata umum)
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence) # Menggabungkan token-token menjadi kalimat

# **Feature Extraction & Labeling**

Labeling terlebih dahulu dilakukan secara manual kemudian menggunakan metode fine-tuning untuk generate label sisanya. Berikut link notebook fine-tuning: [Fine-Tuning Sentiment](https://colab.research.google.com/drive/1a4HLtnXiQys386MTtZ7QuFOKLYR41LQP?usp=sharing)

In [209]:
import os
import urllib.request
import zipfile
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [210]:
# Fungsi untuk mengunduh embedding GloVe jika belum tersedia
def download_glove(dimension=100):
    glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
    zip_path = "glove.6B.zip"

    # Jika file zip belum ada, unduh dari URL
    if not os.path.exists(zip_path):
        urllib.request.urlretrieve(glove_url, zip_path)

    glove_file = f"glove.6B.{dimension}d.txt"
    # Jika file GloVe spesifik dimensi belum ada, ekstrak dari zip
    if not os.path.exists(glove_file):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(glove_file)

    return glove_file

# Fungsi untuk memuat embedding GloVe dari file ke dalam dictionary
def load_glove_file(glove_file):
    embeddings_dict = {}
    try:
        # Buka file dan baca baris per baris
        with open(glove_file, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]  # Kata dari baris
                vector = np.array(values[1:], dtype='float32')  # Vektor numerik dari kata
                embeddings_dict[word] = vector  # Simpan ke dictionary
        return embeddings_dict
    except FileNotFoundError:
        return None

# Fungsi untuk membuat embedding dokumen dengan merata-ratakan vektor kata
def get_doc_embedding(doc, embeddings_dict, embedding_dim):
    words = doc.split()  # Pisahkan dokumen menjadi kata-kata
    doc_embedding = np.zeros(embedding_dim)  # Inisialisasi vektor dokumen dengan nol
    count = 0
    for word in words:
        if word in embeddings_dict:  # Jika kata ditemukan dalam embedding
            doc_embedding += embeddings_dict[word]  # Tambahkan vektor kata ke total
            count += 1
    if count > 0:
        doc_embedding /= count  # Rata-rata jika ada kata yang cocok
    return doc_embedding

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['sentiment']

# Ekstraksi fitur menggunakan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Unduh dan muat embedding GloVe
embedding_dim = 100
glove_file = download_glove(embedding_dim)
embeddings = load_glove_file(glove_file)

# Membuat embedding dokumen menggunakan GloVe
print("Creating document embeddings...")
X_docs = []
for doc in X:
    X_docs.append(get_doc_embedding(doc, embeddings, embedding_dim))
X_glove = np.array(X_docs)

Creating document embeddings...


In [212]:
# Split data hasil Tf-Idf
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [213]:
# Split data hasil GloVe embeddings
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_glove, y, test_size=0.2, random_state=42)

# **Modeling**

### EKSPERIMEN 1:

Pelatihan : SVM

Ekstraksi Fitur : TF-IDF

Pembagian Data : 80/20

In [214]:
# Pelatihan SVM dengan kernel rbf
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)

# Evaluasi pada data pelatihan
y_pred_train = svm.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train, average='weighted')
train_recall = recall_score(y_train, y_pred_train, average='weighted')
train_f1 = f1_score(y_train, y_pred_train, average='weighted')

# Evaluasi pada data pengujian
y_pred_test = svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data = {
    'Accuracy': [train_accuracy, test_accuracy],
    'Precision': [train_precision, test_precision],
    'Recall': [train_recall, test_recall],
    'F1 Score': [train_f1, test_f1]
}

metrics_df = pd.DataFrame(metrics_data, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik SVM:")
metrics_df

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Tabel Evaluasi Metrik SVM:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Accuracy,Precision,Recall,F1 Score
Train,0.981051,0.980083,0.981051,0.980555
Test,0.965547,0.964275,0.965547,0.964884


### EKSPERIMEN 2:

Pelatihan : Random Forest

Ekstraksi Fitur : TF-IDF

Pembagian Data : 70/30

In [215]:
# Pelatihan Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluasi pada data pelatihan
y_pred_train_1 = rf.predict(X_train)
train_accuracy_1 = accuracy_score(y_train, y_pred_train_1)
train_precision_1 = precision_score(y_train, y_pred_train_1, average='weighted')
train_recall_1 = recall_score(y_train, y_pred_train_1, average='weighted')
train_f1_1 = f1_score(y_train, y_pred_train_1, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_1 = rf.predict(X_test)
test_accuracy_1 = accuracy_score(y_test, y_pred_test_1)
test_precision_1 = precision_score(y_test, y_pred_test_1, average='weighted')
test_recall_1 = recall_score(y_test, y_pred_test_1, average='weighted')
test_f1_1 = f1_score(y_test, y_pred_test_1, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_1 = {
    'Accuracy': [train_accuracy_1, test_accuracy_1],
    'Precision': [train_precision_1, test_precision_1],
    'Recall': [train_recall_1, test_recall_1],
    'F1 Score': [train_f1_1, test_f1_1]
}

metrics_df_1 = pd.DataFrame(metrics_data_1, index=['Train', 'Test'])
print("\nTabel Evalusi Metrik Random Forest:")
metrics_df_1


Tabel Evalusi Metrik Random Forest:


,Accuracy,Precision,Recall,F1 Score
Train,0.993719,0.993720,0.993719,0.993678
Test,0.957651,0.956683,0.957651,0.957167


### EKSPERIMEN 3:

Pelatihan : SVM

Ekstraksi Fitur : GloVe

Pembagian Data : 80/20

In [216]:
# Pelatihan SVM dengan kernel rbf
svm_glove = SVC(kernel='rbf')
svm_glove.fit(X_train1, y_train1)

# Evaluasi pada data pelatihan
y_pred_train_2 = svm_glove.predict(X_train1)
train_accuracy_2 = accuracy_score(y_train1, y_pred_train_2)
train_precision_2 = precision_score(y_train1, y_pred_train_2, average='weighted')
train_recall_2 = recall_score(y_train1, y_pred_train_2, average='weighted')
train_f1_2 = f1_score(y_train1, y_pred_train_2, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_2 = svm_glove.predict(X_test1)
test_accuracy_2 = accuracy_score(y_test1, y_pred_test_2)
test_precision_2 = precision_score(y_test1, y_pred_test_2, average='weighted')
test_recall_2 = recall_score(y_test1, y_pred_test_2, average='weighted')
test_f1_2 = f1_score(y_test1, y_pred_test_2, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_2 = {
    'Accuracy': [train_accuracy_2, test_accuracy_2],
    'Precision': [train_precision_2, test_precision_2],
    'Recall': [train_recall_2, test_recall_2],
    'F1 Score': [train_f1_2, test_f1_2]
}

metrics_df_2 = pd.DataFrame(metrics_data_2, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik SVM:")
metrics_df_2

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Tabel Evaluasi Metrik SVM:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Accuracy,Precision,Recall,F1 Score
Train,0.952519,0.951361,0.952519,0.951906
Test,0.934109,0.932797,0.934109,0.933442


### EKSPERIMEN 4:

Pelatihan : Random Forest

Ekstraksi Fitur : GloVe

Pembagian Data : 80/20

In [217]:
# Pelatihan Random Forest
rf_glove = RandomForestClassifier(random_state=42)
rf_glove.fit(X_train1, y_train1)

# Evaluasi pada data pelatihan
y_pred_train_3 = rf_glove.predict(X_train1)
train_accuracy_3 = accuracy_score(y_train1, y_pred_train_3)
train_precision_3 = precision_score(y_train1, y_pred_train_3, average='weighted')
train_recall_3 = recall_score(y_train1, y_pred_train_3, average='weighted')
train_f1_3 = f1_score(y_train1, y_pred_train_3, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_3 = rf_glove.predict(X_test1)
test_accuracy_3 = accuracy_score(y_test1, y_pred_test_3)
test_precision_3 = precision_score(y_test1, y_pred_test_3, average='weighted')
test_recall_3 = recall_score(y_test1, y_pred_test_3, average='weighted')
test_f1_3 = f1_score(y_test1, y_pred_test_3, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_3 = {
    'Accuracy': [train_accuracy_3, test_accuracy_3],
    'Precision': [train_precision_3, test_precision_3],
    'Recall': [train_recall_3, test_recall_3],
    'F1 Score': [train_f1_3, test_f1_3]
}

metrics_df_3 = pd.DataFrame(metrics_data_3, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik Random Forest:")
metrics_df_3


Tabel Evaluasi Metrik Random Forest:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Accuracy,Precision,Recall,F1 Score
Train,0.985824,0.985828,0.985824,0.985716
Test,0.918748,0.916677,0.918748,0.917546


# **Inferensi**

In [218]:
# Input data tanpa label
prediction = pd.read_csv('/content/Maxim-Sentiment-Analysis/maxim_inference.csv')
prediction

,content
0,pengemudi baikudah langgananramahanak aku sena...
1,sangat ramah sekali
2,baik cara mengemudinya bagus
3,saya member maxsim kalau pagi rumah murah kala...
4,lebih baik membantu meringankan seseorangpublik
...,...
71846,kaya tai saya nunggu lama bangget padahal map ...
71847,dulu iya map kurang akurat driver ramah sopan ...
71848,kok gk da aplikasi ny cma play store
71849,kecewa beratttaplagi sm driver kasartolong bina


In [219]:
# Transformasi teks yang sudah dipreproses menggunakan vectorizer TF-IDF yang telah dilatih
X_pred = tfidf.transform(prediction['content'])

# Lakukan prediksi
prediction['sentiment'] = rf.predict(X_pred)
prediction

,content,sentiment
0,pengemudi baikudah langgananramahanak aku sena...,Negatif
1,sangat ramah sekali,Positif
2,baik cara mengemudinya bagus,Positif
3,saya member maxsim kalau pagi rumah murah kala...,Negatif
4,lebih baik membantu meringankan seseorangpublik,Positif
...,...,...
71846,kaya tai saya nunggu lama bangget padahal map ...,Negatif
71847,dulu iya map kurang akurat driver ramah sopan ...,Negatif
71848,kok gk da aplikasi ny cma play store,Negatif
71849,kecewa beratttaplagi sm driver kasartolong bina,Positif


In [226]:
prediction.sentiment.value_counts()

,count
sentiment,
Positif,42752
Negatif,29085
Netral,14
